In [29]:
# imports
# basic
import numpy as np
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_env import TrainState
# for rendering
import PIL
from flatland.utils.rendertools import RenderTool
from IPython.display import clear_output
# for plotting
import matplotlib.pyplot as plt
# for running clingo
import json
import subprocess
import os
import pickle

# Functions

In [30]:
# render the environment
def render_env(env,wait=True):
    
    env_renderer = RenderTool(env, gl="PILSVG")
    env_renderer.render_env()

    image = env_renderer.get_image()
    pil_image = PIL.Image.fromarray(image)
    pil_image = pil_image.convert("RGB")
    clear_output(wait=True)
    display(pil_image)

## Export

In [31]:
# get global transitions
def get_transitions(obsx):
    transitions = obsx[0][0]
    x = 0
    atoms = []

    for i in transitions:
        y = 0
        for j in i:
            directions = ["(\"N\",\"N\")", "(\"N\",\"E\")", "(\"N\",\"S\")", "(\"N\",\"W\")", "(\"E\",\"N\")", "(\"E\",\"E\")", "(\"E\",\"S\")", "(\"E\",\"W\")", "(\"S\",\"N\")", "(\"S\",\"E\")", "(\"S\",\"S\")", "(\"S\",\"W\")", "(\"W\",\"N\")", "(\"W\",\"E\")", "(\"W\",\"S\")", "(\"W\",\"W\")"]
            usableDirections = [directions[i] for i, bit in enumerate(j) if bit == 1]

            for k in usableDirections:
                atoms.append(f"transition(({x},{y}),{k})")
            y += 1
        x += 1

    return atoms

In [32]:
# extract info of an agent
def get_agents(agent):
    atoms = []

    # state(A,(X,Y),T,D)
    # A - agent handle
    # (X,Y) - position of the agent
    # T - time til earliest departure
    # D - direction "N", "E", "S" or "W"
    directions = {0:"\"N\"", 1:"\"E\"", 2:"\"S\"", 3:"\"W\""}
    initialstate = "initialstate(" + str(agent.handle) + ", " + str(agent.initial_position) + ", " + str(agent.earliest_departure) + ", " + str(directions[agent.direction]) + ")"
    atoms.append(initialstate)

    # target(A,(X,Y),T)
    # A - agent handle
    # (X,Y) - position of the target
    # T - time til latest arrival
    target = "target(" + str(agent.handle) + ", "+str(agent.target) + ", " + str(agent.latest_arrival) + ")"
    atoms.append(target)

    # speed(A,S)
    # A - agent handle
    # S - steps taken for one transition
    speed = "speed(" + str(agent.handle) + ", " + str(agent.speed_counter.max_count+1) + ")"
    atoms.append(speed)

    return(atoms)

In [33]:
# get all atoms combined (agents and transitions)
def get_atoms(env,obs,filename):
    # get atoms
    agent_handles = env.get_agent_handles()
    atoms = get_transitions(obs[agent_handles[0]])
    for i in agent_handles:
        atoms = atoms + get_agents(env.agents[i])

    # export atoms
    atoms.append("")
    output = ". ".join(atoms)
    with open(filename, "w") as file:
        file.write(output)

## Import

In [34]:
# inputs are given as file names
def get_orders(input, encoding):
    # Define the command to run
    command = "clingo " + input + " " + encoding + " --outf=2 | jq '.'"

    # Run the command and capture its output
    try:
        output_bytes = subprocess.check_output(command, shell=True)
        output = output_bytes.decode('utf-8')
        data = json.loads(output)
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON: {e}")
        return None

    if data is not None:
        # Extract the "Value" field
        values = data["Call"][0]["Witnesses"][0]["Value"]

        # Initialize a dictionary to store the dictionaries
        dictionaries = {}

        # Iterate through the values
        for value in values:
            parts = value.split(",")
            x = int(parts[0].split("(")[1])
            t = int(parts[1])
            d = parts[2].strip(")\"")

            # Create a dictionary for x if it doesn't exist
            if x not in dictionaries:
                dictionaries[x] = {}

            # Add the corresponding d to the dictionary with t as the key
            dictionaries[x][t] = d

        return dictionaries

    return None

In [35]:
# run the orders
def run_orders(env, orders):
    t = 0

    while True:
        dictionary = {}
        for i in orders:
            if t in orders[i]: value = orders[i][t]
            else: value = 0
            dictionary[i] = int(value)
        print(str(t) + ": "+ str(dictionary))
    
        obs, rew, done, info = env.step(dictionary)
        # render_env(env,True)

        if done["__all__"]:
            return all(info["state"][i] == TrainState.DONE for i in info["state"])

        t += 1


In [36]:
def run_manually(env):
    while True:
        dictionary = {}
        for i in env.get_agent_handles():
            print("Order for agent " + str(i) + ":")
            order = input()
            if order: dictionary[i] = int(order)
            else: dictionary[i] = 0
    
        obs, rew, done, info = env.step(dictionary)
        render_env(env,True)

        if done["__all__"]: break

# Execution

{0: {14: '2', 13: '2'}, 1: {19: '2', 18: '2'}}


In [37]:
filename = "ex24"

with open("testing/flatland/objects/20x24_2/" + filename + ".pkl", "rb") as file:
    env = pickle.load(file)

obs = env.reset()
render_env(env,True)

orders = get_orders("testing/flatland/facts/20x24_2/" + filename + ".lp","encodings/multiplePath/combined.lp")
print(orders)

In [38]:
# run_manually(env)
print(run_orders(env,orders))
# os.remove("encodings/atoms.lp")

0: {0: 0, 1: 0}
1: {0: 0, 1: 0}
2: {0: 0, 1: 0}
3: {0: 0, 1: 0}
4: {0: 0, 1: 0}
5: {0: 0, 1: 0}
6: {0: 0, 1: 0}
7: {0: 0, 1: 0}
8: {0: 0, 1: 0}
9: {0: 0, 1: 0}
10: {0: 0, 1: 0}
11: {0: 0, 1: 0}
12: {0: 0, 1: 0}
13: {0: 2, 1: 0}
14: {0: 2, 1: 0}
15: {0: 0, 1: 0}
16: {0: 0, 1: 0}
17: {0: 0, 1: 0}
18: {0: 0, 1: 2}
19: {0: 0, 1: 2}
20: {0: 0, 1: 0}
21: {0: 0, 1: 0}
22: {0: 0, 1: 0}
23: {0: 0, 1: 0}
24: {0: 0, 1: 0}
25: {0: 0, 1: 0}
26: {0: 0, 1: 0}
27: {0: 0, 1: 0}
28: {0: 0, 1: 0}
29: {0: 0, 1: 0}
30: {0: 0, 1: 0}
31: {0: 0, 1: 0}
32: {0: 0, 1: 0}
33: {0: 0, 1: 0}
34: {0: 0, 1: 0}
35: {0: 0, 1: 0}
36: {0: 0, 1: 0}
37: {0: 0, 1: 0}
38: {0: 0, 1: 0}
39: {0: 0, 1: 0}
40: {0: 0, 1: 0}
41: {0: 0, 1: 0}
42: {0: 0, 1: 0}
True
